<a href="https://colab.research.google.com/github/C4viar/PPD_MSLD_2019_2020_LL-AL-MT/blob/master/Draft_code/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Réplication de l'exemple GIT



## Installation des packages

In [0]:
! pip install transformers

In [0]:
! pip install torchvision

## Imports 

In [0]:
import os 
import torch
import pandas as pd

In [75]:

camembert = torch.hub.load('pytorch/fairseq', 'camembert.v0');
camembert.eval();  # disable dropout (or leave in train mode to finetune)

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master


In [0]:
# Download camembert model
! wget 'https://dl.fbaipublicfiles.com/fairseq/models/camembert.v0.tar.gz' ; 
! tar -xzvf camembert.v0.tar.gz ;

In [0]:
# Load the model in fairseq
from fairseq.models.roberta import CamembertModel
camembert = CamembertModel.from_pretrained('/content/camembert.v0');
camembert.eval();  # disable dropout (or leave in train mode to finetune)

In [78]:
masked_line = 'Le camembert est <mask> :)'
camembert.fill_mask(masked_line, topk=3)


[('Le camembert est délicieux :)', 0.4909120500087738, ' délicieux'),
 ('Le camembert est excellent :)', 0.10556947439908981, ' excellent'),
 ('Le camembert est succulent :)', 0.034533340483903885, ' succulent')]

In [0]:
# Extract the last layer's features
line = "J'aime le camembert !"
tokens = camembert.encode(line)
last_layer_features = camembert.extract_features(tokens)
assert last_layer_features.size() == torch.Size([1, 10, 768])

# Extract all layer's features (layer 0 is the embedding layer)
all_layers = camembert.extract_features(tokens, return_all_hiddens=True)
assert len(all_layers) == 13
assert torch.all(all_layers[-1] == last_layer_features)

# Question 1

1. Reproduire avec l’implémentation
https://huggingface.co/transformers/model_doc/camembert.html
les manipulations décrites dans https://github.com/pytorch/fairseq/tree/master/examples/camembert

## CamemBERT

Rappel : Tous les types de modèles : https://huggingface.co/models

In [0]:
! pip install transformers;

#### Fill mask

In [81]:
from transformers import pipeline, CamembertModel, CamembertTokenizer


sentence = """Le camembert c'est <mask>""".lstrip()

camembert = "camembert-base"

model_camembert = CamembertModel.from_pretrained(camembert)

tokenizer_camembert = CamembertTokenizer.from_pretrained(camembert)


fill_mask = pipeline(
    "fill-mask",
    model = model_camembert,
    tokenizer = tokenizer_camembert
    )

fill_mask(sentence)

[{'score': 0.5193097591400146,
  'sequence': "<s> Le camembert c'est jours</s>",
  'token': 274},
 {'score': 0.003460721345618367,
  'sequence': "<s> Le camembert c'esten</s>",
  'token': 90},
 {'score': 0.001355467364192009,
  'sequence': "<s> Le camembert c'est femme</s>",
  'token': 270},
 {'score': 0.0010079997591674328,
  'sequence': "<s> Le camembert c'estp</s>",
  'token': 286},
 {'score': 0.0009088139049708843,
  'sequence': "<s> Le camembert c'est T</s>",
  'token': 309}]

#### Extract All-layers

In [0]:
input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
labels = torch.tensor([1] * input_ids.size(1)).unsqueeze(0)

In [0]:
outputs_camembert = model(input_ids)

In [85]:
outputs_cammebert

(tensor([[[ 20.6235,  -4.2011,   6.8420,  ...,  -5.7573,  -3.1165,   1.1391],
          [  2.6217,  -5.2430,  17.5290,  ..., -14.2041,  -2.4229,   0.8586],
          [ -0.1691,  -9.8924,   1.8213,  ..., -22.9433, -11.8068,  -1.9409],
          ...,
          [ -0.6569, -10.2129,  -0.5032,  ...,  -7.7102, -10.5380,   1.3649],
          [ -1.6542,  -4.3773,   2.8810,  ...,  -7.5383,  -3.0471,  -3.1600],
          [  6.3686,  -5.1610,  23.3708,  ...,  -7.9306,  -5.1287,   1.9971]]],
        grad_fn=<AddBackward0>),)

## FlauBERT
### Ne fonctionne pas pour le moment

In [0]:
from transformers import FlaubertModel, AutoTokenizer, AutoModelWithLMHead

flaubert = "flaubert-large-cased"

tokenizer_flaubert = AutoTokenizer.from_pretrained(flaubert)
model_flaubert = AutoModelWithLMHead.from_pretrained(flaubert)


fill_mask = pipeline(
    "fill-mask",
    model = flaubert,
    tokenizer = flaubert
)

fill_mask(sentence)

### Flaubert : essai 2

In [0]:

import torch
from transformers import FlaubertModel, FlaubertTokenizer

# Choose among ['flaubert-small-cased', 'flaubert-base-uncased', 'flaubert-base-cased', 'flaubert-large-cased']
modelname = 'flaubert-base-cased' 

# Load pretrained model and tokenizer
flaubert, log = FlaubertModel.from_pretrained(modelname, output_loading_info=True)
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(modelname, do_lowercase=False)
# do_lowercase=False if using cased models, True if using uncased ones

sentence = "Le camembert c'est <mask>."
token_ids = torch.tensor([flaubert_tokenizer.encode(sentence)])

fill_mask = pipeline(
    "fill-mask",
    model = flaubert,
    tokenizer = flaubert_tokenizer
)

fill_mask(sentence)

#last_layer = flaubert(token_ids)[0]
#print(last_layer.shape)
# torch.Size([1, 8, 768])  -> (batch size x number of tokens x embedding dimension)

# The BERT [CLS] token correspond to the first hidden state of the last layer
#cls_embedding = last_layer[:, 0, :]